In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random
import time
from utils import epoch_time, count_parameters, categorical_accuracy

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = False

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.long)

In [2]:
fields = [('text', TEXT), ('label', LABEL)]

train_data, test_data = data.TabularDataset.splits(
                                        path = 'data',
                                        train = 'trial.tsv',
                                        test = 'test.tsv',
                                        format = 'tsv',
                                        fields = fields,
                                        skip_header = True
)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [3]:
vars(train_data[-1])

{'text': ['Rest',
  'from',
  'your',
  'worries',
  'found',
  'here',
  '.',
  '|',
  ':',
  '@user',
  '#',
  'tnstateparks',
  '@',
  'Radnor',
  'Lake',
  'Natural',
  '…'],
 'label': '10'}

In [4]:
for d in train_data:
    if len(vars(d)['text']) < 2:
        print(vars(d))

{'text': ['Happy'], 'label': '0'}
{'text': ['@userPhilly'], 'label': '0'}
{'text': ['Bar+me+drinks='], 'label': '2'}
{'text': ['Lmfaoooooooooooo'], 'label': '2'}
{'text': ['omar'], 'label': '2'}
{'text': ['LMFAO'], 'label': '2'}


In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [6]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f7259646d08>, {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '14': 13, '13': 14, '17': 15, '16': 16, '15': 17, '18': 18, '19': 19})


In [7]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    sort_key=lambda x: len(x.text),
    batch_size = BATCH_SIZE, 
    device = device)

In [8]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved if conv.shape[2] != 0]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [9]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [10]:
model.convs

ModuleList(
  (0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
  (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
  (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
)

In [11]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,596,520 trainable parameters


In [12]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.6610, -0.0730,  0.9238,  ..., -0.2256,  0.8148, -0.4405],
        ...,
        [ 0.1128,  2.0460,  0.8266,  ...,  0.7674, -0.5384, -1.8739],
        [ 1.7783,  1.1309,  0.0594,  ..., -0.2007, -0.7234,  0.1532],
        [ 0.0433,  0.3691, -2.7507,  ...,  1.7293, -1.0242,  0.0297]])

In [13]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Train model

In [14]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [15]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            if batch.text.shape[0] < 4:
                continue
            
            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
model

CNN(
  (embedding): Embedding(25002, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  )
  (fc): Linear(in_features=300, out_features=20, bias=True)
  (dropout): Dropout(p=0.5)
)

In [18]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epoch: 01 | Epoch Time: 0m 19s
	Train Loss: 2.665 | Train Acc: 23.04%
	 Val. Loss: 2.502 |  Val. Acc: 25.90%
Epoch: 02 | Epoch Time: 0m 20s
	Train Loss: 2.474 | Train Acc: 27.30%
	 Val. Loss: 2.429 |  Val. Acc: 27.46%
Epoch: 03 | Epoch Time: 0m 21s
	Train Loss: 2.352 | Train Acc: 30.55%
	 Val. Loss: 2.394 |  Val. Acc: 29.04%
Epoch: 04 | Epoch Time: 0m 20s
	Train Loss: 2.234 | Train Acc: 33.55%
	 Val. Loss: 2.372 |  Val. Acc: 29.43%
Epoch: 05 | Epoch Time: 0m 20s
	Train Loss: 2.127 | Train Acc: 36.07%
	 Val. Loss: 2.374 |  Val. Acc: 29.55%
